# Drawing App with AI Painting Conversion

This notebook demonstrates how to create a simple drawing application using PyQt5 and convert the drawing into a famous painting using a pre-trained model from Hugging Face. The application will allow users to draw stick figures and see them transformed into paintings.


## Step 1: Import Libraries

We need to import the necessary libraries for our application.


In [1]:
from PyQt5.QtWidgets import QApplication, QMainWindow, QVBoxLayout, QPushButton, QLabel, QWidget, QHBoxLayout, QColorDialog
from PyQt5.QtGui import QPainter, QPen, QPixmap, QImage
from PyQt5.QtCore import Qt, QPoint
from PIL import Image
from io import BytesIO
import torch
from diffusers import StableDiffusionImg2ImgPipeline
import sys

## Step 2: Create the Drawing Canvas

We will create a `Canvas` class that allows users to draw stick figures. The canvas will support drawing with a pen and erasing with an eraser.


In [2]:
class Canvas(QWidget):
    def __init__(self):
        super().__init__()
        self.setFixedSize(800, 500)
        self.image = QPixmap(self.size())
        self.image.fill(Qt.white)
        self.drawing = False
        self.last_point = QPoint()
        self.eraser_mode = False
        self.pen_color = Qt.black  # Default pen color

    def paintEvent(self, event):
        canvas_painter = QPainter(self)
        canvas_painter.drawPixmap(self.rect(), self.image, self.image.rect())

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            self.drawing = True
            self.last_point = event.pos()

    def mouseMoveEvent(self, event):
        if event.buttons() & Qt.LeftButton and self.drawing:
            painter = QPainter(self.image)
            if self.eraser_mode:
                painter.setPen(QPen(Qt.white, 20, Qt.SolidLine, Qt.RoundCap, Qt.RoundJoin))
            else:
                painter.setPen(QPen(self.pen_color, 4, Qt.SolidLine, Qt.RoundCap, Qt.RoundJoin))
            painter.drawLine(self.last_point, event.pos())
            self.last_point = event.pos()
            self.update()

    def mouseReleaseEvent(self, event):
        if event.button() == Qt.LeftButton:
            self.drawing = False

    def save_image(self, path):
        self.image.save(path)

    def load_image(self, qimage):
        self.image = QPixmap.fromImage(qimage)
        self.update()

    def clear_canvas(self):
        self.image.fill(Qt.white)
        self.update()

    def toggle_eraser(self):
        self.eraser_mode = not self.eraser_mode

    def set_pen_color(self, color):
        self.pen_color = color

## Step 3: Create the Main Application Window

We will create a `DrawingApp` class that includes the drawing canvas and a button to convert the drawing to a painting.


In [3]:
class DrawingApp(QMainWindow):
    def __init__(self):
        super().__init__()
        self.initUI()
        self.generated_image = None

    def initUI(self):
        self.setWindowTitle('Drawing App')
        self.setGeometry(100, 100, 800, 800)

        self.canvas = Canvas()
        main_layout = QVBoxLayout()
        main_layout.addWidget(self.canvas)

        self.painting_label = QLabel(self)
        main_layout.addWidget(self.painting_label)

        btn_layout = QHBoxLayout()
        convert_btn = QPushButton('Convert to Painting', self)
        convert_btn.clicked.connect(self.convert_to_painting)
        btn_layout.addWidget(convert_btn)

        clear_btn = QPushButton('Clear Canvas', self)
        clear_btn.clicked.connect(self.canvas.clear_canvas)
        btn_layout.addWidget(clear_btn)

        eraser_btn = QPushButton('Eraser', self)
        eraser_btn.setCheckable(True)
        eraser_btn.clicked.connect(self.canvas.toggle_eraser)
        btn_layout.addWidget(eraser_btn)

        color_btn = QPushButton('Select Color', self)
        color_btn.clicked.connect(self.select_color)
        btn_layout.addWidget(color_btn)

        copy_btn = QPushButton('Copy AI Image to Canvas', self)
        copy_btn.clicked.connect(self.copy_image_to_canvas)
        btn_layout.addWidget(copy_btn)

        main_layout.addLayout(btn_layout)

        container = QWidget()
        container.setLayout(main_layout)
        self.setCentralWidget(container)

    def convert_to_painting(self):
        self.canvas.save_image('stick_figure.png')
        image, original_size = preprocess_image('stick_figure.png')
        painting = convert_to_painting(image)
        painting = painting.resize(original_size)
        self.generated_image = painting  # Store the generated image
        painting_qimage = self.pil_image_to_qimage(painting)
        self.painting_label.setPixmap(QPixmap.fromImage(painting_qimage))

    def copy_image_to_canvas(self):
        if self.generated_image:
            painting_qimage = self.pil_image_to_qimage(self.generated_image)
            self.canvas.load_image(painting_qimage)

    def pil_image_to_qimage(self, pil_image):
        byte_data = BytesIO()
        pil_image.save(byte_data, format="PNG")
        q_image = QImage()
        q_image.loadFromData(byte_data.getvalue())
        return q_image

    def select_color(self):
        color = QColorDialog.getColor()
        if color.isValid():
            self.canvas.set_pen_color(color)

## Step 4: Preprocess the Image

If necessary, we might need to preprocess the drawn image before sending it to the AI model.


In [4]:
def preprocess_image(image_path):
    image = Image.open(image_path).convert("RGB")
    original_size = image.size
    return image, original_size


## Step 5: Convert the Image to a Painting

We will use a pre-trained model from Hugging Face to convert the drawing to a painting.


In [5]:
def convert_to_painting(image):
    # Ensure the image is in the correct format (PIL.Image)
    if not isinstance(image, Image.Image):
        raise ValueError("The input image must be a PIL.Image")

    # Debug: Print the type of the image to confirm it's a PIL.Image
    print(f"Type of image inside convert_to_painting: {type(image)}")

    # Load a pre-trained image-to-image translation model
    model_id = "stabilityai/stable-diffusion-2"
    pipe = StableDiffusionImg2ImgPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")
    prompt = "A beautiful painting in the style of Van Gogh."

    result = pipe(prompt=prompt, image=image, strength=0.6, guidance_scale=7.5)
    return result.images[0]

# # test the function
# image = Image.open('stick_figure.png')
# resized_image, original_size = preprocess_image('stick_figure.png')
# painting = convert_to_painting(resized_image)
# painting.show()

## Step 6: Run the Application

Finally, we will run the PyQt application within the Jupyter Notebook.


In [6]:
def run_app():
    app = QApplication.instance()
    if app is None:
        app = QApplication(sys.argv)
    window = DrawingApp()
    window.show()
    
    app.exec_()


# Run the application
run_app()
